In [3]:
import pandas as pd
import numpy as np
from collections import Counter
from copy import copy
from numpy import genfromtxt
from scipy import spatial
from numpy import dot
from numpy.linalg import norm

In [4]:
df = pd.read_csv("Superpowers.csv")
netid = list(df["Cornell Tech Email (Required)"].apply(lambda x: x.split("@")[0]))
name  = list(df["Slack (Full Name) (Required)"])
name_2_netid = {n:id for n, id in zip(name, netid)}
netid_2_name = {id:n for n, id in zip(name, netid)}
index_2_name = {i:n for i, n in enumerate(name)}
name_2_index = {n:i for i, n in enumerate(name)}

In [5]:
# read data
categorical_data = genfromtxt('./cleaned_data/categorical_data.csv', delimiter=',')[1:]
bow_data = genfromtxt('./cleaned_data/bow_data.csv', delimiter=',')

data = np.concatenate((categorical_data, bow_data), axis=1)
data.shape

(350, 621)

In [6]:
# cosine similarity
cos_sim = lambda a, b: dot(a, b)/(norm(a)*norm(b))
cos_sim(data[100], data[200])

0.3326495719295578

In [7]:
# jaccard similarity
def jaccard_sim(x, y):
	intersection = np.logical_and(x, y)
	union = np.logical_or(x, y)
	similarity = intersection.sum() / float(union.sum())
	return similarity

jaccard_sim(data[100], data[200])

0.1978021978021978

In [24]:
def mehh(data, sim, n, name):
	ret = []
	for i,u1 in enumerate(data):
		u1_friends = { j:sim(u1,u2) if i!=j else 0 for j, u2 in enumerate(data) }

		# list friends based on the sim values
		u1_bff = [ (k,v) for k, v in sorted(u1_friends.items(), key=lambda item: item[1], reverse=True) ][:n]
		ret.append(u1_bff)
	
	data = []
	for idx, score in ret[name_2_index[name]]:
		name = index_2_name[idx]
		data.append([name, score])
	
	return pd.DataFrame(data, columns=["Name", "Score"])



In [26]:
mehh(data=data, sim=cos_sim, n=15, name="Ethan Huang, CS")

,Name,Score
0,Evan Lin,0.446594
1,Zhengxing Xue,0.439388
2,Richa Vishwakarma,0.422561
3,"Jarrett Coleman, CS",0.410136
4,Rajesh Mudidana,0.405798
5,"Joan La Rosa, CS",0.398314
6,Yang Song,0.397796
7,Ridhima Sakhuja,0.392913
8,"Jinxu Liu, CM",0.392466
9,"Crystal Tang, CS",0.379193
